In [ ]:
from transformers import pipeline

classifier = pipeline(model="IDEA-CCNL/Erlangshen-TCBert-110M-Classification-Chinese", task="sentiment-analysis")

classifier("我今天还是觉得高兴，因为这两天学习了不少新的技能。")


In [ ]:
# Try zero-shot-classification pipeline

from transformers import pipeline

classifier = pipeline("zero-shot-classification")

classifier(
    "This is Rafael ZHU, and I like the course about the Transformers library from HuggingFace community!",
    candidate_labels=["education", "politics", "business", "sports"],
)

In [ ]:
# Try text generation

from transformers import pipeline

generator = pipeline(task="text-generation", model="distilgpt2")

generator("In this course, I will learn ", max_length=30, num_return_sequences=2)

In [ ]:
# Try mask filling

from transformers import pipeline

unmasker = pipeline("fill-mask")

unmasker("This course will teach you all about <mask> models, and practise real skills.", top_k=3)



In [ ]:
# Try some Named Entity Recognition (NER) task

from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)

ner("My name is Sylvian and I worked at Hugging Face in Brooklyn, and Nadal today annouced the quit of Austrilia Open, so sad news.")

In [ ]:
# Try some question answering 

from transformers import pipeline

qa = pipeline("question-answering", model="deepset/roberta-base-squad2")

qa(
    question="How old am I?",
    context="My name is Rafael, and I was born in Dec 1988, Zhejiang province of China."
  )

In [ ]:
# Try text summarization

from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

In [ ]:
# Install missing package of sentencepiece

! pip install -U sentencepiece

In [1]:
# Try translation from French to English

from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

translator("Ce cours est produit par Hugging Face.")

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'This course is produced by Hugging Face.'}]

In [2]:
# Attention the limits and bias

from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


In [6]:
# Deep dive into pipeline of sentiment-analysis

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    
    "I have been waiting for HuggingFace course my whole life.",
    "I hate this so much, it's dangerous at all.",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

print(inputs)

from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

outputs = model(**inputs)

print(outputs.last_hidden_state.shape)

{'input_ids': tensor([[  101,  1045,  2031,  2042,  3403,  2005, 17662, 12172,  2607,  2026,
          2878,  2166,  1012,   102,     0],
        [  101,  1045,  5223,  2023,  2061,  2172,  1010,  2009,  1005,  1055,
          4795,  2012,  2035,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 15, 768])


In [12]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs = model(**inputs)

print(outputs.logits)

import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

print(predictions)


tensor([[-2.3642,  2.4149],
        [ 3.8902, -3.2022]], grad_fn=<AddmmBackward0>)
tensor([[8.3338e-03, 9.9167e-01],
        [9.9917e-01, 8.3069e-04]], grad_fn=<SoftmaxBackward0>)


In [13]:
# Initialize a BERT model

from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [16]:
# Load a pretrained model and save it to S3 bucket for future usage

from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")


# Get the s3 bucket 

import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

# Save model to s3 bucket
#model.save_pretrained(f"s3://{sess.default_bucket()}/dl24-data/bert-base-cased")

model.save_pretrained("temp/bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
import os

print(os.getcwd())

/home/ec2-user/SageMaker


In [19]:
# Try with some Tokenizer details

tokenized_text = "Jim Henson's son was a puppeteer !".split()

print(tokenized_text)

['Jim', "Henson's", 'son', 'was', 'a', 'puppeteer', '!']


In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer("Using a Transformer network is simple")

tokenizer.save_pretrained("temp/bert-tokenizer")


('temp/bert-tokenizer/tokenizer_config.json',
 'temp/bert-tokenizer/special_tokens_map.json',
 'temp/bert-tokenizer/vocab.txt',
 'temp/bert-tokenizer/added_tokens.json',
 'temp/bert-tokenizer/tokenizer.json')

In [29]:
# Step by step of tokenzier, get the tokens first

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple. So I'll keep learning @ 2024."
tokens = tokenizer.tokenize(sequence)

print(tokens)


# Then translate the tokens into ids
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

# Decode part of the id sequences into text again.
decoded_string = tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 3014, 119, 1573])

print(decoded_string)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple', '.', 'So', 'I', "'", 'll', 'keep', 'learning', '@', '202', '##4', '.']
[7993, 170, 13809, 23763, 2443, 1110, 3014, 119, 1573, 146, 112, 1325, 1712, 3776, 137, 17881, 1527, 119]
Using a Transformer network is simple. So
